In [47]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
import statistics 
from sklearn.neighbors import KNeighborsClassifier
from operator import itemgetter

## Upload dan Membaca File

In [48]:
#upload file 
from google.colab import files
uploaded = files.upload()

Saving Diabetes.csv to Diabetes (3).csv


In [49]:
#membaca file
import io
dataset = pd.read_csv(io.StringIO(uploaded['Diabetes.csv'].decode('utf-8')))

## Prapemrosesan Data

In [50]:
#Preprocessing
X = dataset.iloc[:, :8]
y = dataset['Outcome'].values

In [51]:
#input mising value
X = X.replace(0, np.nan)

imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean')
imp_mean.fit(X)
X = imp_mean.transform(X)

In [52]:
#Membagi data training set dan testing set

#case data ke-1
trainingset_X = X[:614]
trainingset_y = y[:614]
testingset_X = X[614:]
testingset_y = y[614:]

# #case data ke-2
trainingset_X = np.concatenate((X[:461], X[614:]))
trainingset_y = np.concatenate((y[:461], y[614:]))
testingset_X = X[461:614]
testingset_y = y[461:614]

# case data ke-3
trainingset_X = np.concatenate((X[:307], X[461:]))
trainingset_y = np.concatenate((y[:307], y[461:]))
testingset_X = X[307:461]
testingset_y = y[307:461]

#case data ke-4
trainingset_X = np.concatenate((X[:154], X[307:]))
trainingset_y = np.concatenate((y[:154], y[307:]))
testingset_X = X[154:307]
testingset_y = y[154:307]

#case data ke-5
trainingset_X = X[155:]
trainingset_y = y[155:]
testingset_X = X[:155]
testingset_y = y[:155]



In [ ]:
error_rate = []
for i in range(1,40):
 knn = KNeighborsClassifier(n_neighbors=i)
 knn.fit(trainingset_X,trainingset_y)
 pred_i = knn.predict(testingset_X)
 error_rate.append(np.mean(pred_i != testingset_y))

plt.figure(figsize=(10,6))
plt.plot(range(1,40),error_rate,color='blue', linestyle='dashed', 
         marker='o',markerfacecolor='red', markersize=10)
plt.title('Error Rate vs. K Value')
plt.xlabel('K')
plt.ylabel('Error Rate')
print("Minimum error:-",min(error_rate),"at K =",error_rate.index(min(error_rate)))


## Perhitungan Ukuran Jarak

In [54]:
def manhattan(trainingset,testingset):
  return np.sum(np.abs(np.array(trainingset)-np.array(testingset)))

#eucliden
# def eucliden(trainingset,testingset):
#   return np.sqrt(np.sum((trainingset-testingset)**2))

# a = [100, 200]
# b = [400, 600]
# manhattan(a, b)


## Klasifikasi KNN

In [64]:
#Klasifikasi KNN
def klasifikasi(k):
  hasil = []
  nt = []
  hasil_pembelajaran = []
  for i in testingset_X:
    hasil = []
    for j in trainingset_X:
      hasil.append(manhattan(i,j))
      sort = np.argsort(hasil)[:k]
    
    for l in sort:
      nt.append(trainingset_y[l])
    hasil_pembelajaran.append(statistics.mode(nt))
    nt = [] 
  return hasil_pembelajaran

## Pemilihan Rata-Rata Akurasi KNN Menggunakan 5-fold cross-validation dan Pemilihan Nilai K Terbaik


In [57]:
#5-fold Cross-Validation
k = 3
nk = []
for i in range(20):
  total_akurasi = 0
  for j in range(5):
    if j == 0:
      trainingset_X = X[:614]
      trainingset_y = y[:614]
      testingset_X = X[614:]
      testingset_y = y[614:]
    elif j == 1:
      trainingset_X = np.concatenate((X[:461], X[614:]))
      trainingset_y = np.concatenate((y[:461], y[614:]))
      testingset_X = X[461:614]
      testingset_y = y[461:614]
    elif j == 2:
      trainingset_X = np.concatenate((X[:307], X[461:]))
      trainingset_y = np.concatenate((y[:307], y[461:]))
      testingset_X = X[307:461]
      testingset_y = y[307:461]
    elif j == 3:
      trainingset_X = np.concatenate((X[:154], X[307:]))
      trainingset_y = np.concatenate((y[:154], y[307:]))
      testingset_X = X[154:307]
      testingset_y = y[154:307]
    elif j == 4:
       trainingset_X = X[155:]
       trainingset_y = y[155:]
       testingset_X = X[:155]
       testingset_y = y[:155]

#Scaling
    scaler=StandardScaler()
    trainingset_X = scaler.fit_transform(trainingset_X)
    testingset_X = scaler.transform(testingset_X)

#Pemilihan nilai rata-rata akurasi 
    total_akurasi += (np.sum(klasifikasi(k) == testingset_y) / len(testingset_y))*100
  print(i+1, 'Rata-rata akurasi dengan nilai K',k,':' ,total_akurasi/5)
#Pemilihan nilai K terbaik
  nk.append([total_akurasi/5, k])
  k = k+2
  nk = sorted(nk, key=itemgetter(0), reverse=True)
print('Dan K =', nk[0][1], 'merupakan K yang memiliki nilai akurasi yang tertinggi yaitu:', nk[0][0])

1 Rata-rata akurasi dengan nilai K 3 : 72.81768073798435
2 Rata-rata akurasi dengan nilai K 5 : 73.99661017877337
3 Rata-rata akurasi dengan nilai K 7 : 73.21150239176805
4 Rata-rata akurasi dengan nilai K 9 : 75.29115497616446
5 Rata-rata akurasi dengan nilai K 11 : 76.1977213172659
6 Rata-rata akurasi dengan nilai K 13 : 76.5865047876433
7 Rata-rata akurasi dengan nilai K 15 : 77.3708185131335
8 Rata-rata akurasi dengan nilai K 17 : 77.88523894406248
9 Rata-rata akurasi dengan nilai K 19 : 77.11190517262622
10 Rata-rata akurasi dengan nilai K 21 : 76.71636396494083
11 Rata-rata akurasi dengan nilai K 23 : 75.80387775833697
12 Rata-rata akurasi dengan nilai K 25 : 75.9337478882071
13 Rata-rata akurasi dengan nilai K 27 : 75.93207214459588
14 Rata-rata akurasi dengan nilai K 29 : 75.93122332021764
15 Rata-rata akurasi dengan nilai K 31 : 76.32338018296272
16 Rata-rata akurasi dengan nilai K 33 : 76.32338018296272
17 Rata-rata akurasi dengan nilai K 35 : 76.97697495420455
18 Rata-rata a